# Create database; scrape data; store scraped data and state population data as two tables

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
from time import sleep
from random import randint
import pandas as pd

In [4]:
def create_storage():
    """This function creates our Metal_Bands db and Band_Info table."""
    
    # connect to sqlite
    conn = sqlite3.connect('Metal_Bands.db')
    cursor = conn.cursor()

    # create band_info table
    cursor.execute("""CREATE TABLE BAND_INFO(
                        Band_Name text,
                        Status text,
                        Location text,
                        Year_Formed text,
                        Genre text,
                        Lyrical_Themes text,
                        Years_Active text)""")
    conn.commit()
    conn.close()

In [5]:
def scrape(url):
    """This function takes the given url and iterates through each tab of bands, scraping 
        each band page for its data. The data is stored in the Metal_Bands database."""
    
    # connect to sqlite db
    conn = sqlite3.connect('Metal_Bands.db')
    cursor = conn.cursor()
    
    # loop over each tab of band links
    for i in range(72):
        # get html of tab webpage (session deals with cookies?)
        session = requests.Session()
        response = session.get(url, 
                           headers={'User-Agent': 'Mozilla/5.0'})

        # "prettify" html code
        soup = BeautifulSoup(response.content, "html.parser")
        
        # get all the band links
        links = [link.get("href") for link in soup.find_all("a")]
            
        # rest
        sleep(randint(3,5))
            
        # scrape each link
        for j in range(len(links)):
            
            # get html of band page
            session = requests.Session()
            response = session.get(links[j], headers={'User-Agent': 'Mozilla/5.0'})

            # "prettify" html code
            soup = BeautifulSoup(response.content, "html.parser")
            
            # get band data
            band_name = soup.select("h1.band_name")[0].get_text()
            band_info = soup.select("div#band_stats")[0].get_text()
            
            location = re.search(r"(?<=Location:\n).*", band_info).group(0)
            status = re.search(r"(?<=Status:\n).*", band_info).group(0)
            year_formed = re.search(r"(?<=Formed in:\n).*", band_info).group(0)
            genre = re.search(r"(?<=Genre:\n).*", band_info).group(0)
            lyrical_themes = re.search(r"(?<=Lyrical themes:\n).*", band_info).group(0)
            years_active = band_info.split(":")[-1].strip("\n").strip()
            
            # insert data into sqlite db
            row = (band_name, status, location, year_formed, genre, lyrical_themes, years_active)
            cursor.execute("INSERT INTO BAND_INFO VALUES (?,?,?,?,?,?,?)", row)
            conn.commit()
            
            # rest
            sleep(randint(3,5))
            
        # update url to be that of next tab
        echo = re.search(r"(?<=Echo=)\d+", url)
        echo = str(int(echo.group(0)) + 1)
        url = re.sub(r"(?<=Echo=)\d+", echo, url)
        
        disp_start = re.search(r"(?<=iDisplayStart=)\d+", url)
        disp_start = str(int(disp_start.group(0)) + 500)
        url = re.sub(r"(?<=iDisplayStart=)\d+", disp_start, url)

        last_arg = re.search(r"\d+$", url)
        last_arg = str(int(last_arg.group(0)) + 1)
        url = re.sub(r"\d+$", last_arg, url)
        
        # indicate which tabs are complete
        print("Tab done:", i)
        
    # close connection to db
    conn.close()        

In [6]:
# run em... don't do this again!
url = "https://www.metal-archives.com/browse/ajax-country/c/US/json/1?sEcho=1&iColumns=4&sColumns=&iDisplayStart=0&iDisplayLength=500&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&mDataProp_3=3&iSortCol_0=0&sSortDir_0=asc&iSortingCols=1&bSortable_0=true&bSortable_1=true&bSortable_2=true&bSortable_3=false&_=1668986854743"
#create_storage()
#scrape(url)

Tab done: 0
Tab done: 1
Tab done: 2
Tab done: 3
Tab done: 4
Tab done: 5
Tab done: 6
Tab done: 7
Tab done: 8
Tab done: 9
Tab done: 10
Tab done: 11
Tab done: 12
Tab done: 13
Tab done: 14
Tab done: 15
Tab done: 16
Tab done: 17
Tab done: 18
Tab done: 19
Tab done: 20
Tab done: 21
Tab done: 22
Tab done: 23
Tab done: 24
Tab done: 25
Tab done: 26
Tab done: 27
Tab done: 28
Tab done: 29
Tab done: 30
Tab done: 31
Tab done: 32
Tab done: 33
Tab done: 34
Tab done: 35
Tab done: 36
Tab done: 37
Tab done: 38
Tab done: 39
Tab done: 40
Tab done: 41
Tab done: 42
Tab done: 43
Tab done: 44
Tab done: 45
Tab done: 46
Tab done: 47
Tab done: 48
Tab done: 49
Tab done: 50
Tab done: 51
Tab done: 52
Tab done: 53
Tab done: 54
Tab done: 55
Tab done: 56
Tab done: 57
Tab done: 58
Tab done: 59
Tab done: 60
Tab done: 61
Tab done: 62
Tab done: 63
Tab done: 64
Tab done: 65
Tab done: 66
Tab done: 67
Tab done: 68
Tab done: 69
Tab done: 70
Tab done: 71


In [16]:
# check if worked...DONE!
conn = sqlite3.connect('Metal_Bands.db')
cursor = conn.cursor()

data = cursor.execute("SELECT * FROM BAND_INFO")
#conn.close()
i = 0
num_rows = 0
for row in data:
    if i < 100:
        print(row)
    i += 1
    num_rows += 1
conn.close()
print(num_rows)

('۞', 'Split-up', 'N/A', 'N/A', 'Black Metal', 'N/A', 'N/A')
('...and Here I Lie', 'Split-up', 'Holley, New York', '1994', 'Death/Doom Metal', 'Depression, Isolation, Suicide', '1994-2000,\n2007')
('...and They Will Meet God', 'Split-up', 'Palestine, Texas', '2002', 'Melodic Death Metal/Crossover', 'Doom, Death, Hate', '2002-?')
('...in Agony of the Eclipsed Moon', 'Active', 'Midland City, Alabama', 'N/A', 'Black Metal/Dark Ambient', 'N/A', 'N/A')
('...Under a Full Moon', 'Active', 'Midland, Michigan (early); North Carolina, United States / Norway (later)', '2017', 'Black/Doom Metal', 'Depression, Suicide, Anti-capitalism', '2016-2017  (as Disrhythmia),\n2017-present')
('.357', 'Split-up', 'Jackson, Mississippi', '2004', 'Black Metal/Grindcore', 'N/A', '2004-2006')
('.44 Caliber Killers', 'Unknown', 'Syracuse, New York', '2001', 'Thrash Metal/Punk/Crossover', 'Flaws, Serial killers, Pizza, Politics, Humour', '2001-?')
('.50Cal Facial Fracture', 'Active', 'Philadelphia, Pennsylvania', '

In [16]:
# need to load state pop data into db now
# from: https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html#par_textimage_1574439295
state_data = pd.read_csv("State_Pops.csv")
state_data.set_index("State", inplace=True)

conn = sqlite3.connect("Metal_Bands.db")
cursor = conn.cursor()
state_data.to_sql(name='STATE_INFO', con=conn, index_label='State')
conn.close()

In [17]:
# check if worked...DONE!
conn = sqlite3.connect('Metal_Bands.db')
cursor = conn.cursor()

data = cursor.execute("SELECT * FROM STATE_INFO")
i = 0
num_rows = 0
for row in data:
    if i < 100:
        print(row)
    i += 1
    num_rows += 1
conn.close()
print(num_rows)

('Alabama', '5,039,877')
('Alaska', '732,673')
('Arizona', '7,276,316')
('Arkansas', '3,025,891')
('California', '39,237,836')
('Colorado', '5,812,069')
('Connecticut', '3,605,597')
('Delaware', '1,003,384')
('Florida', '21,781,128')
('Georgia', '10,799,566')
('Hawaii', '1,441,553')
('Idaho', '1,900,923')
('Illinois', '12,671,469')
('Indiana', '6,805,985')
('Iowa', '3,193,079')
('Kansas', '2,934,582')
('Kentucky', '4,509,394')
('Louisiana', '4,624,047')
('Maine', '1,372,247')
('Maryland', '6,165,129')
('Massachusetts', '6,984,723')
('Michigan', '10,050,811')
('Minnesota', '5,707,390')
('Mississippi', '2,949,965')
('Missouri', '6,168,187')
('Montana', '1,104,271')
('Nebraska', '1,963,692')
('Nevada', '3,143,991')
('New Hampshire', '1,388,992')
('New Jersey', '9,267,130')
('New Mexico', '2,115,877')
('New York', '19,835,913')
('North Carolina', '10,551,162')
('North Dakota', '774,948')
('Ohio', '11,780,017')
('Oklahoma', '3,986,639')
('Oregon', '4,246,155')
('Pennsylvania', '12,964,056')

In [19]:
state_data["Population"]

State
Alabama            5,039,877
Alaska               732,673
Arizona            7,276,316
Arkansas           3,025,891
California        39,237,836
Colorado           5,812,069
Connecticut        3,605,597
Delaware           1,003,384
Florida           21,781,128
Georgia           10,799,566
Hawaii             1,441,553
Idaho              1,900,923
Illinois          12,671,469
Indiana            6,805,985
Iowa               3,193,079
Kansas             2,934,582
Kentucky           4,509,394
Louisiana          4,624,047
Maine              1,372,247
Maryland           6,165,129
Massachusetts      6,984,723
Michigan          10,050,811
Minnesota          5,707,390
Mississippi        2,949,965
Missouri           6,168,187
Montana            1,104,271
Nebraska           1,963,692
Nevada             3,143,991
New Hampshire      1,388,992
New Jersey         9,267,130
New Mexico         2,115,877
New York          19,835,913
North Carolina    10,551,162
North Dakota         774,948
Ohio    